# Run Expectancy

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 999

In [2]:
fields = pd.read_csv('../data/fields.csv', index_col=0)
data2016 = pd.read_csv('../data/all2016.csv', names=fields.Header)

In [3]:
data2016['RUNS'] = data2016.AWAY_SCORE_CT + data2016.HOME_SCORE_CT
data2016['HALF_INNING'] = (data2016.GAME_ID 
                            + data2016.INN_CT.astype(str) 
                            + data2016.BAT_HOME_ID.astype(str))
data2016['RUNS_SCORED'] = ((data2016.BAT_DEST_ID > 3).astype(int) 
                            + (data2016.RUN1_DEST_ID > 3).astype(int)
                            + (data2016.RUN2_DEST_ID > 3).astype(int)
                            + (data2016.RUN3_DEST_ID > 3).astype(int))

In [5]:
half_innings = data2016.groupby('HALF_INNING').agg(outs_inning=('EVENT_OUTS_CT',sum),
                                                    runs_inning=('RUNS_SCORED',sum),
                                                    runs_start=('RUNS',min))
half_innings['MAX_RUNS'] = half_innings.runs_inning + half_innings.runs_start

The runs from the event to the end of the inning is the difference between the total runs scored in the inning and the runs scored up until the event.

In [8]:
data2016 = data2016.join(half_innings, on='HALF_INNING', how='inner')
data2016['RUNS_ROI'] = data2016.MAX_RUNS - data2016.RUNS

In [16]:
assert not data2016.loc[data2016.MAX_RUNS < data2016.RUNS, ['HALF_INNING', 'RUNS_SCORED', 'RUNS', 'MAX_RUNS']].values

## Create Matrix

Create string variable for base occupancy state.

In [10]:
data2016['STATE'] = ((~data2016.BASE1_RUN_ID.isnull()).astype(int).astype(str)
                    + (~data2016.BASE2_RUN_ID.isnull()).astype(int).astype(str)
                    + (~data2016.BASE3_RUN_ID.isnull()).astype(int).astype(str)
                    + data2016.OUTS_CT.astype(str))

data2016['STATE'].unique()

array(['0000', '0100', '0011', '0012', '1002', '0001', '0002', '1001',
       '1000', '1100', '1101', '0111', '1011', '1102', '0110', '1111',
       '1012', '1112', '0102', '1010', '0101', '0112', '1110', '0010'],
      dtype=object)

Create indicators for base occupancy following the play.

In [11]:
data2016['NRUNNER1'] = ((data2016.RUN1_DEST_ID==1) | (data2016.BAT_DEST_ID==1)).astype(int)
data2016['NRUNNER2'] = ((data2016.RUN1_DEST_ID==2) | (data2016.RUN2_DEST_ID==2) 
                            | (data2016.BAT_DEST_ID==2)).astype(int)
data2016['NRUNNER3'] = ((data2016.RUN1_DEST_ID==3) | (data2016.RUN2_DEST_ID==3) 
                            | (data2016.RUN3_DEST_ID==3) | (data2016.BAT_DEST_ID==3)).astype(int)
data2016['NOUTS'] = data2016.OUTS_CT + data2016.EVENT_OUTS_CT
data2016['NEW_BASES'] = data2016[['NRUNNER1', 'NRUNNER2', 'NRUNNER3']].astype(int).astype(str).agg(''.join, axis=1)
data2016['NEW_STATE'] = data2016[['NEW_BASES', 'NOUTS']].astype(str).agg(''.join, axis=1)

data2016[['STATE', 'NEW_STATE']].head()

,STATE,NEW_STATE
0,0000,0100
1,0100,0011
2,0011,0012
3,0012,1002
4,1002,1003


Condition on change between state and new state, or runs scored.

In [12]:
data2016 = data2016.query('(STATE != NEW_STATE) | (RUNS_SCORED > 0)')

Remove incomplete innings.

In [13]:
data2016_complete = data2016.query('outs_inning==3')

Calculate run expectancy for base/outs states.

In [23]:
expected_runs = data2016_complete.groupby('STATE').agg(mean_runs=('RUNS_ROI', 'mean')).reset_index()
expected_runs['base_state'] = expected_runs.STATE.str[:3]
expected_runs['outs'] = expected_runs.STATE.str[-1]

In [28]:
expected_runs.pivot(index='base_state', columns='outs', values='mean_runs').round(2)

outs,0,1,2
base_state,,,
000,0.50,0.27,0.11
001,1.35,0.94,0.37
010,1.13,0.67,0.31
011,1.93,1.36,0.55
100,0.86,0.51,0.22
101,1.72,1.20,0.48
110,1.44,0.92,0.41
111,2.11,1.54,0.70


## Estimating the success of a batting play